In [1]:
#configure the necessary Spark environment
import os, sys

#del os.environ['PYSPARK_SUBMIT_ARGS']

#here you should set where are located the spark files
spark_home = '/home/vagrant/spark-2.0.2-bin-hadoop2.7/'

os.environ["SPARK_HOME"]  = spark_home

sys.path.insert(0, spark_home + "/python")

sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.3-src.zip'))

#this defines the sparkContext sc
execfile(os.path.join(spark_home, 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.0.2
      /_/

Using Python version 2.7.12 (default, Jul  1 2016 15:12:24)
SparkSession available as 'spark'.


In [2]:
%matplotlib inline
import datetime
import numpy as np

/usr/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
interactions_file = sc.textFile('../data/interactions.psv')
interactions_header = interactions_file.first()

products_file = sc.textFile('../data/products.psv')
products_header = products_file.first()

interactions_by_product = (interactions_file
                .filter(lambda l: l != interactions_header)
                .map(lambda l: l.split("|"))
                .map(lambda l: (l[2], [l[0], l[1], l[3], l[4], datetime.datetime.fromtimestamp(int(l[5])/1000.0).replace(minute=0, hour=0, second=0, microsecond=0) ]))
                .filter(lambda l: l[1][1] in [u'viewed', u'bought'])
)
products = (products_file
            .filter(lambda l: l != products_header)
            .map(lambda l: l.split("|"))
            .map(lambda l: (l[0], [l[1], l[2], l[3], l[4], float(l[5])  ]))
)

In [4]:
users_product = (interactions_by_product
     .map(lambda x: (x[1][0], x[0]))
     .groupByKey()
     .map(lambda x : (x[0], list(set(x[1]))))
)

In [5]:
product_product = (interactions_by_product
                            .map(lambda x: (x[1][0], x[0]))
                            .join(users_product)
                            .map(lambda x: (x[1][0], x[1][1]))
                            .reduceByKey(lambda a, b: a+b)
                            .map(lambda x : (x[0], list(set(x[1]))))
                            .collect()
)

In [6]:
product_suggestion = sc.parallelize(product_product)
product_suggestion.take(10)

[(u'd44a37d5-e07d-4668-bf30-fa73dd51ee19',
  [u'd44a37d5-e07d-4668-bf30-fa73dd51ee19']),
 (u'd4fe2d2f-42ef-4498-a739-7eb9a158bb45',
  [u'd4fe2d2f-42ef-4498-a739-7eb9a158bb45']),
 (u'a7a270e8-eaa0-4186-ae8e-ceec368a7da2',
  [u'a7a270e8-eaa0-4186-ae8e-ceec368a7da2']),
 (u'afd05e52-f1b4-4c3b-a83b-5a31b6a9e518',
  [u'afd05e52-f1b4-4c3b-a83b-5a31b6a9e518']),
 (u'300b6541-8d6b-48ad-9647-f3a7d4d007e9',
  [u'300b6541-8d6b-48ad-9647-f3a7d4d007e9']),
 (u'd71a2931-ef52-48ca-9297-5522146f1b12',
  [u'd71a2931-ef52-48ca-9297-5522146f1b12']),
 (u'b9906424-7ade-4f56-ba31-e0dae557aca2',
  [u'b9906424-7ade-4f56-ba31-e0dae557aca2',
   u'32754f51-f1da-4e47-9568-06b374c65c69']),
 (u'62abe1df-e519-494a-911b-e3357fe11af1',
  [u'62abe1df-e519-494a-911b-e3357fe11af1']),
 (u'26a7069e-ab50-4b83-b1e1-ac44b3defc74',
  [u'26a7069e-ab50-4b83-b1e1-ac44b3defc74',
   u'4364723c-6a9b-48df-b36f-6ae6c8cf00d5',
   u'865a4ee3-7cc2-4da5-b444-440209add9f6',
   u'50127342-17b5-48d9-8bf2-acfeef846857']),
 (u'8f821e84-9661-4cec-